# SPY Spreadsheet

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import itable

import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'TSLA'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [5]:
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2020-10-12,448.74,438.58,442.00,442.30,38791100.00,442.30
2020-10-13,448.89,436.60,443.35,446.65,34463700.00,446.65
2020-10-14,465.90,447.35,449.78,461.30,48045400.00,461.30
2020-10-15,456.57,442.50,450.31,448.88,35672400.00,448.88
2020-10-16,455.95,438.85,454.44,439.67,32620000.00,439.67


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [6]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [7]:
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
2010-06-29,5.00,3.51,3.80,4.78,93831500.00,4.78
2010-06-30,6.08,4.66,5.16,4.77,85935500.00,4.77
2010-07-01,5.18,4.05,5.00,4.39,41094000.00,4.39
2010-07-02,4.62,3.74,4.60,3.84,25699000.00,3.84
2010-07-06,4.00,3.17,4.00,3.22,34334500.00,3.22


Add technical indicator: 200 day MA

In [8]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [9]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2020-10-12,448.74,438.58,442.00,442.30,38791100.00,442.30,221.44
2020-10-13,448.89,436.60,443.35,446.65,34463700.00,446.65,223.24
2020-10-14,465.90,447.35,449.78,461.30,48045400.00,461.30,225.13
2020-10-15,456.57,442.50,450.31,448.88,35672400.00,448.88,226.96
2020-10-16,455.95,438.85,454.44,439.67,32620000.00,439.67,228.73


Add technical indicator: ATR

In [10]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [11]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr
date,,,,,,,,
2020-10-12,448.74,438.58,442.00,442.30,38791100.00,442.30,221.44,25.79
2020-10-13,448.89,436.60,443.35,446.65,34463700.00,446.65,223.24,24.82
2020-10-14,465.90,447.35,449.78,461.30,48045400.00,461.30,225.13,24.42
2020-10-15,456.57,442.50,450.31,448.88,35672400.00,448.88,226.96,24.02
2020-10-16,455.95,438.85,454.44,439.67,32620000.00,439.67,228.73,23.53


Add technical indicator: 5 day high, and 5 day low

In [12]:
high5 = pd.Series(ts.high).rolling(window=5).max()
low5 = pd.Series(ts.low).rolling(window=5).min()
ts['high5'] = high5
ts['low5'] = low5

In [13]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2020-10-12,448.74,438.58,442.00,442.30,38791100.00,442.30,221.44,25.79,448.74,406.05
2020-10-13,448.89,436.60,443.35,446.65,34463700.00,446.65,223.24,24.82,448.89,413.85
2020-10-14,465.90,447.35,449.78,461.30,48045400.00,461.30,225.13,24.42,465.90,425.30
2020-10-15,456.57,442.50,450.31,448.88,35672400.00,448.88,226.96,24.02,465.90,426.46
2020-10-16,455.95,438.85,454.44,439.67,32620000.00,439.67,228.73,23.53,465.90,436.60


Add technical indicator: RSI, and 2-period cumulative RSI

In [14]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.Series(ts.rsi2).rolling(window=2).sum()
ts['c2rsi2'] = c2rsi2

In [15]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2020-10-12,448.74,438.58,442.00,442.30,38791100.00,442.30,221.44,25.79,448.74,406.05,91.99,175.00
2020-10-13,448.89,436.60,443.35,446.65,34463700.00,446.65,223.24,24.82,448.89,413.85,94.84,186.83
2020-10-14,465.90,447.35,449.78,461.30,48045400.00,461.30,225.13,24.42,465.90,425.30,98.48,193.33
2020-10-15,456.57,442.50,450.31,448.88,35672400.00,448.88,226.96,24.02,465.90,426.46,44.83,143.31
2020-10-16,455.95,438.85,454.44,439.67,32620000.00,439.67,228.73,23.53,465.90,436.60,24.79,69.62


Add technical indicator: Midpoint

In [16]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [17]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2020-10-12,448.74,438.58,442.00,442.30,38791100.00,442.30,221.44,25.79,448.74,406.05,91.99,175.00,443.66
2020-10-13,448.89,436.60,443.35,446.65,34463700.00,446.65,223.24,24.82,448.89,413.85,94.84,186.83,442.75
2020-10-14,465.90,447.35,449.78,461.30,48045400.00,461.30,225.13,24.42,465.90,425.30,98.48,193.33,456.62
2020-10-15,456.57,442.50,450.31,448.88,35672400.00,448.88,226.96,24.02,465.90,426.46,44.83,143.31,449.54
2020-10-16,455.95,438.85,454.44,439.67,32620000.00,439.67,228.73,23.53,465.90,436.60,24.79,69.62,447.40


Add technical indicator: SMA10 of midpoint

In [18]:
sma10mp = pd.Series(ts.mp).rolling(window=10).mean()
ts['sma10mp'] = sma10mp

In [19]:
ts.head(10)

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
2010-06-29,5.00,3.51,3.80,4.78,93831500.00,4.78,nan,nan,nan,nan,nan,nan,4.25,nan
2010-06-30,6.08,4.66,5.16,4.77,85935500.00,4.77,nan,nan,nan,nan,nan,nan,5.37,nan
2010-07-01,5.18,4.05,5.00,4.39,41094000.00,4.39,nan,nan,nan,nan,0.00,nan,4.62,nan
2010-07-02,4.62,3.74,4.60,3.84,25699000.00,3.84,nan,nan,nan,nan,0.00,0.00,4.18,nan
2010-07-06,4.00,3.17,4.00,3.22,34334500.00,3.22,nan,nan,6.08,3.17,0.00,0.00,3.58,nan
2010-07-07,3.33,3.00,3.28,3.16,34608500.00,3.16,nan,nan,6.08,3.00,0.00,0.00,3.16,nan
2010-07-08,3.50,3.11,3.23,3.49,38557000.00,3.49,nan,nan,5.18,3.00,54.37,54.37,3.31,nan
2010-07-09,3.58,3.31,3.52,3.48,20253000.00,3.48,nan,nan,4.62,3.00,52.31,106.68,3.44,nan
2010-07-12,3.61,3.40,3.59,3.41,11012500.00,3.41,nan,nan,4.00,3.00,36.30,88.61,3.51,nan


Add technical indicator: Standard Deviation

In [20]:
sd = pd.Series(ts.mp).rolling(window=10).std()
ts['sd'] = sd

In [21]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd
date,,,,,,,,,,,,,,,
2020-10-12,448.74,438.58,442.00,442.30,38791100.00,442.30,221.44,25.79,448.74,406.05,91.99,175.00,443.66,428.81,8.60
2020-10-13,448.89,436.60,443.35,446.65,34463700.00,446.65,223.24,24.82,448.89,413.85,94.84,186.83,442.75,431.08,9.01
2020-10-14,465.90,447.35,449.78,461.30,48045400.00,461.30,225.13,24.42,465.90,425.30,98.48,193.33,456.62,434.02,11.94
2020-10-15,456.57,442.50,450.31,448.88,35672400.00,448.88,226.96,24.02,465.90,426.46,44.83,143.31,449.54,434.81,12.73
2020-10-16,455.95,438.85,454.44,439.67,32620000.00,439.67,228.73,23.53,465.90,436.60,24.79,69.62,447.40,436.84,12.98


In [22]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [23]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2020-10-12,448.74,438.58,442.00,442.30,38791100.00,442.30,221.44,25.79,448.74,406.05,91.99,175.00,443.66,428.81,8.60,446.00,411.61
2020-10-13,448.89,436.60,443.35,446.65,34463700.00,446.65,223.24,24.82,448.89,413.85,94.84,186.83,442.75,431.08,9.01,449.10,413.05
2020-10-14,465.90,447.35,449.78,461.30,48045400.00,461.30,225.13,24.42,465.90,425.30,98.48,193.33,456.62,434.02,11.94,457.89,410.15
2020-10-15,456.57,442.50,450.31,448.88,35672400.00,448.88,226.96,24.02,465.90,426.46,44.83,143.31,449.54,434.81,12.73,460.27,409.35
2020-10-16,455.95,438.85,454.44,439.67,32620000.00,439.67,228.73,23.53,465.90,436.60,24.79,69.62,447.40,436.84,12.98,462.80,410.89


Select a smaller time from for use with itable

In [24]:
df = ts['2019-01-01':]

In [25]:
df.head()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-01-02,63.03,59.76,61.22,62.02,58293000.00,62.02,62.49,3.71,67.84,58.82,20.18,90.28,61.39,64.28,2.66,69.60,58.95
2019-01-03,61.88,59.48,61.40,60.07,34826000.00,60.07,62.47,3.62,67.84,59.48,12.51,32.69,60.68,63.49,2.42,68.32,58.66
2019-01-04,63.60,60.55,61.20,63.54,36970500.00,63.54,62.48,3.62,67.84,59.48,62.76,75.27,62.07,62.93,1.94,66.81,59.05
2019-01-07,67.35,63.55,64.34,66.99,37756000.00,66.99,62.50,3.63,67.84,59.48,82.64,145.40,65.45,63.05,2.07,67.19,58.92
2019-01-08,68.80,65.40,68.39,67.07,35042500.00,67.07,62.53,3.61,68.80,59.48,83.05,165.69,67.10,63.41,2.43,68.27,58.54


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [26]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme='theme1'), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [27]:
pt

,HIGH,LOW,OPEN,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,UPPER,LOWER
2019/01/02,63.03,59.76,61.22,62.02,58293000,62.02,62.49,3.71,67.84,58.82,20.18,90.28,61.39,64.28,2.66,69.60,58.95
2019/01/03,61.88,59.48,61.40,60.07,34826000,60.07,62.47,3.62,67.84,59.48,12.51,32.69,60.68,63.49,2.42,68.32,58.66
2019/01/04,63.60,60.55,61.20,63.54,36970500,63.54,62.48,3.62,67.84,59.48,62.76,75.27,62.07,62.93,1.94,66.81,59.05
2019/01/07,67.35,63.55,64.34,66.99,37756000,66.99,62.50,3.63,67.84,59.48,82.64,145.40,65.45,63.05,2.07,67.19,58.92
2019/01/08,68.80,65.40,68.39,67.07,35042500,67.07,62.53,3.61,68.80,59.48,83.05,165.69,67.10,63.41,2.43,68.27,58.54
2019/01/09,68.70,66.29,67.10,67.71,27164500,67.71,62.56,3.53,68.80,59.48,87.75,170.80,67.50,64.06,2.58,69.22,58.90
2019/01/10,69.08,66.36,66.88,68.99,30282000,68.99,62.60,3.47,69.08,60.55,94.23,181.98,67.72,64.62,2.71,70.05,59.19
2019/01/11,69.68,67.75,68.42,69.45,25195500,69.45,62.67,3.36,69.68,63.55,95.81,190.04,68.72,65.25,2.87,70.99,59.52
2019/01/14,68.50,66.80,68.48,66.88,26236500,66.88,62.75,3.31,69.68,65.40,23.54,119.34,67.65,65.47,2.97,71.41,59.54
2019/01/15,69.76,66.90,67.00,68.89,30283000,68.89,62.83,3.28,69.76,66.29,64.87,88.41,68.33,65.66,3.09,71.85,59.48
